In [1]:
import os
import sys
import pickle
import pandas as pd

In [2]:
def read_data(filename):
    # endpoint = os.getenv('S3_ENDPOINT_URL', None)
    endpoint = "http://localhost:4566"

    if endpoint is None:
        df = pd.read_parquet(filename)
        return df
    else:
        options = {
            'client_kwargs': {
                'endpoint_url': endpoint
            }
        }

        df = pd.read_parquet(filename, storage_options=options)
        return df

In [18]:
def prepare_data(df, categorical):
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    return df

In [19]:
def get_input_path(year, month):
    default_input_pattern = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
    input_pattern = "s3://nyc-duration/in/yellow_tripdata_{year:04d}-{month:02d}.parquet"
    return input_pattern.format(year=year, month=month)

In [20]:
def get_output_path(year, month):
    default_output_pattern = 's3://nyc-duration-prediction-alexey/taxi_type=fhv/year={year:04d}/month={month:02d}/predictions.parquet'
    output_pattern = "s3://nyc-duration/out/yellow_tripdata_{year:04d}-{month:02d}.parquet"
    return output_pattern.format(year=year, month=month)

In [21]:
def main(year, month):

    # input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
    # output_file = f'yellow_tripdata_{year:04d}-{month:02d}.parquet'

    with open('model.bin', 'rb') as f_in:
        dv, lr = pickle.load(f_in)

    categorical = ['PULocationID', 'DOLocationID']

    df = read_data(input_file)
    df = prepare_data(df, categorical)

    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    y_pred = lr.predict(X_val)

    print('predicted mean duration:', y_pred.mean())

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['predicted_duration'] = y_pred

    df_result.to_parquet(output_file, engine='pyarrow', index=False)

In [22]:
year = 2023
month = 3
input_file = get_input_path(year, month)
output_file = get_output_path(year, month)
main(year, month)

AttributeError: 'DataFrame' object has no attribute 'tpep_dropoff_datetime'

In [23]:
get_input_path(year, month)

's3://nyc-duration/in/yellow_tripdata_2023-03.parquet'

In [3]:
read_data('s3://nyc-duration/out/yellow_tripdata_2023-03.parquet')

,ride_id,predicted_duration
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486
...,...,...
3316211,2023/03_3403761,11.952635
3316212,2023/03_3403762,20.049958
3316213,2023/03_3403763,11.595336
3316214,2023/03_3403764,13.113178
